In [1]:
import warnings
warnings.filterwarnings("ignore")
import requests
import folium
from pandas.io.json import json_normalize

In [2]:
# 전국 또는 특정 지역의 스타벅스 매장 위치를 찾아와서 지도위에 표시하기
# 1. requests로 스타벅스 매장 위치 데이터를 가져와서 딕셔너리 타입으로 변환한다.
# 2. json_normalize() 메소드로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# 3. folium 모듈을 사용해서 지도를 표시하고 지도위의 스타벅스 매장에 마커를 표시한다.

In [4]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=R66KCOMOB6'
request = requests.post(targetSite, data={
    'ins_lat'   : 37.5108295,
    'ins_lng'   : 127.02928809999999,
    'p_sido_cd' : '01',
    'p_gugun_cd': '',
    'in_biz_cd' : '',
    'iend'      : 1600,
    'set_date'  : ''
})
storeList = request.json()
print(len(storeList['list']))

544


In [5]:
star_json = request.json() # json 타입의 문자열을 딕셔너리로 변환한다.
print(type(star_json))     # <class 'dict'>
print(len(star_json))      # 1
print(star_json)

<class 'dict'>
1
{'list': [{'seq': 0, 'sido_cd': None, 'sido_nm': None, 'gugun_cd': None, 'gugun_nm': None, 'code_order': None, 'view_yn': None, 'store_num': None, 'sido': None, 'gugun': None, 'address': None, 'new_img_nm': None, 'p_pro_seq': 0, 'p_view_yn': None, 'p_sido_cd': '', 'p_gugun_cd': '', 'p_store_nm': None, 'p_theme_cd': None, 'p_wireless_yn': None, 'p_smoking_yn': None, 'p_book_yn': None, 'p_music_yn': None, 'p_terrace_yn': None, 'p_table_yn': None, 'p_takeout_yn': None, 'p_parking_yn': None, 'p_dollar_assent': None, 'p_card_recharge': None, 'p_subway_yn': None, 'stb_store_file_renew': None, 'stb_store_theme_renew': None, 'stb_store_time_renew': None, 'stb_store_lsm': None, 's_code': '1509', 's_name': '역삼아레나빌딩', 'tel': '1522-3232', 'fax': '02-568-3763', 'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 'addr': '서울특별시 강남구 역삼동 721-13 아레나빌딩', 'park_info': None, 'new_state': None, 'theme_state': 'T04@T05@T08@T16@T17@T20@T21@T30@T35@T36@P80@P90', '

In [8]:
# pandas.io.json 모듈의 json_normalize() 메소드로 json 타입이 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(데이터프레임으로 변환할 데이터가 저장된 딕셔너리, '딕셔너리 중에서 데이터프레임으로 변환할 키')
star_df = json_normalize(star_json, 'list')
print(type(star_df))
print(len(star_df))

<class 'pandas.core.frame.DataFrame'>
544


In [10]:
star_df

seq sido_cd sido_nm gugun_cd gugun_nm code_order view_yn store_num  sido  \
0      0    None    None     None     None       None    None      None  None   
1      0    None    None     None     None       None    None      None  None   
2      0    None    None     None     None       None    None      None  None   
3      0    None    None     None     None       None    None      None  None   
4      0    None    None     None     None       None    None      None  None   
..   ...     ...     ...      ...      ...        ...     ...       ...   ...   
539    0    None    None     None     None       None    None      None  None   
540    0    None    None     None     None       None    None      None  None   
541    0    None    None     None     None       None    None      None  None   
542    0    None    None     None     None       None    None      None  None   
543    0    None    None     None     None       None    None      None  None   

    gugun  ... p40 p80  t22 t21 p90 t05 t30 t36 t27 t43  
0    None  ...   0   0    0   0   0   0   0   0   0   0  
1    None  ...   0   0    0   0   0   0   0   0   0   0  
2    None  ...   0   0    0   0   0   0   0   0   0   0  
3    None  ...   0   0    0   0   0   0   0   0   0   0  
4    None  ...   0   0    0   0   0   0   0   0   0   0  
..    ...  ...  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  
539  None  ...   0   0    0   0   0   0   0   0   0   0  
540  None  ...   0   0    0   0   0   0   0   0   0   0  
541  None  ...   0   0    0   0   0   0   0   0   0   0  
542  None  ...   0   0    0   0   0   0   0   0   0   0  
543  None  ...   0   0    0   0   0   0   0   0   0   0  

[544 rows x 126 columns]

In [12]:
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name => 지점이름
# sido_code => 시도 코드
# sido_name => 시도 이름
# gugun_code => 구군 코드
# gugun_name => 구군 이름
# doro_address => 도로명 주소
# lat => 위도
# lot => 경도

# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적으면 되지만 얻어올 컬럼이 2개
# 이상일 경우 얻어올 컬럼 이름을 리스트로 만들어 얻어와야 한다.
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map

s_name sido_code sido_name gugun_code gugun_name  \
0    역삼아레나빌딩        01        서울       0101        강남구   
1     논현역사거리        01        서울       0101        강남구   
2    신사역성일빌딩        01        서울       0101        강남구   
3     국기원사거리        01        서울       0101        강남구   
4     스탈릿대치R        01        서울       0101        강남구   
..       ...       ...       ...        ...        ...   
539     사가정역        01        서울       0125        중랑구   
540      상봉역        01        서울       0125        중랑구   
541    묵동이마트        01        서울       0125        중랑구   
542       묵동        01        서울       0125        중랑구   
543      중화역        01        서울       0125        중랑구   

                                    doro_address                lat  \
0                        서울특별시 강남구 언주로 425 (역삼동)          37.501087   
1                       서울특별시 강남구 강남대로 538 (논현동)          37.510178   
2                       서울특별시 강남구 강남대로 584 (논현동)          37.514132   
3                       서울특별시 강남구 테헤란로 125 (역삼동)          37.499517   
4                     서울특별시 강남구 남부순환로 2947 (대치동)          37.494668   
..                                           ...                ...   
539                            서울특별시 중랑구 면목로 310          37.579594   
540              서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)           37.59689   
541  서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)          37.613433   
542                           서울특별시 중랑구 동일로 952           37.615368   
543                       서울특별시 중랑구 봉화산로 35 1,2층  37.60170912407773   

                    lot  
0            127.043069  
1            127.022223  
2            127.020563  
3            127.031495  
4            127.062583  
..                  ...  
539          127.087966  
540           127.08647  
541          127.077484  
542          127.076633  
543  127.07841136432036  

[544 rows x 8 columns]

In [13]:
star_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [14]:
# astype() 메소드로 위도, 경도의 데이터 타입을 object에서 float롤 변경한다.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object

In [15]:
# 종각점
star_df_map[star_df_map['s_name'] == '종각']

s_name sido_code sido_name gugun_code gugun_name            doro_address  \
401     종각        01        서울       0118        종로구  서울특별시 종로구 종로 64 (종로2가)   

           lat         lot  
401  37.569918  126.984528

In [17]:
# 광교점
star_df_map[star_df_map['s_name'] == '종로관철']

s_name sido_code sido_name gugun_code gugun_name  \
403   종로관철        01        서울       0118        종로구   

                     doro_address        lat         lot  
403  서울특별시 종로구 종로12길 21, 2층 (관철동)  37.569058  126.986013